In [1]:
import json
import collections
import numpy as np

In [2]:
f = open('formulas_dict.json', 'r')
formulas_dict = json.load(f)
f.close()
f = open('constants_dict.json', 'r')
constants_dict = json.load(f)
f.close()
f = open('input_variables.json', 'r')
input_variables = json.load(f)
f.close()

## Get dependencies and find a good computation order

### Get dependecies

In [3]:
def get_children(node):
    nodetype = node['nodetype']

    if nodetype == 'symbol':
        name = node['name']
        return set([name])
        
    elif nodetype == 'float':
        return set()
    
    elif nodetype == 'call':
        args = node['args']
        children = set()
        for arg in args:
            children = children | get_children(arg)

        return children
        
    raise ValueError('Unknown type : %s'%nodetype)

In [4]:
children_dict = {}
for name, formula in formulas_dict.items():
    children_dict[name] = get_children(formula)

In [5]:
children_dict['NRILMJV']

{'AILMJV',
 'NRCELTOT',
 'NRILMIA',
 'NRILMIB',
 'NRILMIC',
 'NRILMIH',
 'NRILMIX',
 'NRILMIY',
 'NRILMIZ',
 'NRILMJC',
 'NRILMJI',
 'NRILMJS',
 'NRILMJW',
 'NRILMJX',
 'NRILMJY',
 'NRILMPA',
 'NRILMPB',
 'NRILMPC',
 'NRILMPD',
 'NRILMPE',
 'NRINVRED',
 'NRLOGDOM',
 'NRMEUBLE',
 'NRPROREP',
 'NRREDMEUB',
 'NRREDREP',
 'NRREPMEU',
 'NRREPNPRO',
 'NRRI2',
 'RRI1'}

### Find dependencies without formula

In [6]:
input_list = []
for e in input_variables:
    input_list.append(e['alias'])
    input_list.append(e['name'])
    

In [7]:
unknown_variables = set()
for k, v in children_dict.items():
    for var in v:
        if (var not in formulas_dict) and (var not in constants_dict) and (var not in input_list):
            unknown_variables.add(var)

In [8]:
len(unknown_variables)

1535

### Parents

In [9]:
parents_dict = {}

for k in children_dict:
    parents_dict[k] = set()

for parent, children in children_dict.items():
    for child in children:
        if child in children_dict:
            parents_dict[child].add(parent)

### Dependencies

In [ ]:
iinet_deps = []


### Find cycles

In [16]:
index = {}
reverse_index = []
for i, k in enumerate(children_dict):
    index[k] = i
    reverse_index.append(k)

In [17]:
n = len(children_dict)
adj = np.zeros((n, n), dtype=np.int8)

In [18]:
for parent, children in children_dict.items():
    for child in children:
        if child in children_dict:
            adj[index[parent], index[child]] = 1

In [ ]:
current_power = 1
while current_power <= n:
    print(current_power)
    adj = np.dot(adj, adj)
    current_power *= 2

In [19]:
np.diag(adj).sum()

19

In [21]:
np.where(np.diag(adj))

(array([  667,  3118,  3936,  5007,  5152,  5741,  6456,  6950,  7037,
         8775,  9236,  9971, 10323, 10560, 11510, 11803, 13127, 13589, 13650]),)

In [22]:
reverse_index[667]

'TISF4BASE'

In [23]:
formulas_dict['TISF4BASE']

{'args': [{'args': [{'args': [{'name': 'FLAG_RETARD', 'nodetype': 'symbol'}],
     'name': 'positif',
     'nodetype': 'call'},
    {'args': [{'args': [{'name': 'ISFTARDIF_A', 'nodetype': 'symbol'},
        {'args': [{'nodetype': 'float', 'value': 1.0},
          {'args': [{'args': [{'name': 'FLAG_NBSTRTR', 'nodetype': 'symbol'}],
             'name': 'positif',
             'nodetype': 'call'}],
           'name': '-',
           'nodetype': 'call'}],
         'name': '+',
         'nodetype': 'call'}],
       'name': '*',
       'nodetype': 'call'},
      {'args': [{'name': 'TISF4BASE', 'nodetype': 'symbol'},
        {'args': [{'name': 'FLAG_NBSTRTR', 'nodetype': 'symbol'}],
         'name': 'positif',
         'nodetype': 'call'}],
       'name': '*',
       'nodetype': 'call'}],
     'name': '+',
     'nodetype': 'call'}],
   'name': '*',
   'nodetype': 'call'},
  {'args': [{'args': [{'nodetype': 'float', 'value': 1.0},
      {'args': [{'args': [{'name': 'FLAG_RETARD', 'nodetype': 

### Find good computing order

In [ ]:
# doesn't work : too much recursion
def find_order(node):
    if node not in simple_dependencies_dict:
        return
        
    deps = simple_dependencies_dict[node]
    is_leaf = True
    for dep in deps:
        if dep not in computing_order:
            find_order(dep)
            
    computing_order.append(node)
            

In [ ]:
root = 'IINET'
computing_order = []

to_inspect = [root]
while to_inspect:
    node = to_inspect[-1]
    is_leaf = True

    if node in simple_dependencies_dict:
        deps = simple_dependencies_dict[node]
    
        for dep in deps:
            if dep not in computing_order:
                is_leaf = False
                if dep not in to_inspect:
                    to_inspect.append(dep)
            
    if is_leaf:
        computing_order.append(node)
        to_inspect.pop()



In [ ]:
len(to_inspect)

In [ ]:
find_order(root)

In [ ]:
computing_order

In [ ]:
simple_dependencies_dict[root]

## Compute

In [ ]:
def product(l):
    accu = 1.
    for e in l:
        accu *= e
    return accu

def boolean_or(l):
    for e in l:
        if l:
            return 1.
    return 0.

def boolean_et(l):
    for e in l:
        if not l:
            return 0.
    return 1.

functions_mapping = {
    '+': sum,
    '*': product,
    '-': (lambda x: -x[0]),
    'positif': (lambda x: float(x[0]>0)),
    'positif_ou_nul': (lambda x: float(x[0]>=0)),
    'null': (lambda x: float(x[0]==0)),
    'operator:>=': (lambda x: float(x[0]>=x[1])),
    'operator:=': (lambda x: float(x[0]==x[1])),
    'ternary': (lambda x: x[1] if x[0] else x[2]),
    '/': (lambda x: x[0]/x[1] if x[1] else 0.),
    'max': max,
    'min': min,
    'inf': (lambda x: math.floor(x[0])),
    'arr': (lambda x: round(x[0])),
    'abs': (lambda x: abs(x[0])),
    'present': (lambda x: x[0] != 0.),
    'boolean:ou': boolean_or,
    'boolean:et': boolean_et,
    'dans': (lambda x: 1. if (x[0] in x[1:]) else 0.)
}


In [ ]:
def get_value(name):
    if name not in known_values:
        if name not in formulas_dict:
            print('Variable %s not found !'%name)
            known_values[name] = 0.
        else:
            formula = formulas_dict[name]
            known_values[name] = compute(formula)
            print('%d values known'%len(known_values))
    return known_values[name]

In [ ]:
def compute(node):
    nodetype = node['nodetype']

    if nodetype == 'symbol':
        name = node['name']
        value = get_value(name)
        return value 
        
    elif nodetype == 'float':
        value = node['value']
        return value
    
    elif nodetype == 'call':
        name = node['name']
        args = [compute(child) for child in node['args']]
        function = functions_mapping[name]
        try:
            value = function(args)
        except TypeError:
            print(args)
            print(function)
            raise TypeError()
        return value
        
    raise ValueError('Unknown type : %s'%nodetype)

In [ ]:
saisie = {'TSHALLOV': 30000.}
# http://www3.finances.gouv.fr/calcul_impot/2015/index.htm

known_values = {}

for c in constants:
    name = c['name']
    value = c['value']
    known_values[name] = value

for var in input_variables:
    name = var['name']
    alias = var['alias']
    
    known_alias = (alias in saisie)
    known_name = (name in saisie)
    
    if known_alias and known_name:
        raise ValueError('Variable defined twice.')
    if known_alias:
        value = saisie[alias]
        known_values[alias] = value
    else:
        known_values[alias] = 0.
        
    if known_name:
        value = saisie[name]
        known_values[name] = value
    else:
        known_values[name] = 0.
